In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import numpy as np

# Loading datasets 

In [2]:
cmu_movie_cols = ['wikiID',
                  'freebase_movID',
                  'movie_name',
                  'release_date',
                  'boxoffice',
                  'runtime',
                  'language',
                  'country',
                  'genre']

cmu_character_cols = ['wikiID',
                      'freebase_movID',
                      'release_date',
                      'character_name',
                      'actor_birth',
                      'actor_gender',
                      'actor_height',
                      'actor_ethnicity',
                      'actor_name',
                      'actor_age_at_release',
                      'freebase_char_actor_map',
                      'freebase_charID',
                      'freebase_actorID']

acl_persona_cols = ['entity_id',
                    'wikiID',
                    'movie_name',
                    'entity_name',
                    'full_entity_name',
                    'entity_num_event',
                    'persona',
                    'final_posterior']

data_path = './data/'
acl_persona_path = data_path+'ACL_personas_dump/personas/'

imdb_movies = pd.read_csv(data_path+'title_basics.tsv', sep='\t',low_memory=False)
imdb_ratings= pd.read_csv(data_path+'title_ratings.tsv', sep='\t')
imdb_principals = pd.read_csv(data_path+'name_principals.csv')
cmu_movies = pd.read_csv(data_path+'movie.metadata.tsv', sep='\t', names=cmu_movie_cols)
cmu_character = pd.read_table(data_path+'character.metadata.tsv', names=cmu_character_cols)
acl_persona = pd.read_table(acl_persona_path+'25.100.lda.log.txt', header=None, names=acl_persona_cols)

In [4]:
#merge with their ratings
imdb_movies = imdb_movies[['tconst','primaryTitle','startYear']].merge(imdb_ratings,how='inner',on='tconst')

#lower(title)
imdb_movies = imdb_movies[imdb_movies['primaryTitle'].apply(lambda x : type(x))==str]
imdb_movies['primaryTitle'] = imdb_movies['primaryTitle'].apply(lambda x : x.lower())

imdb_movies = imdb_movies.sort_values(by=['primaryTitle','numVotes'],ascending=[True,False])

imdb_movies = imdb_movies.drop_duplicates(subset=['primaryTitle','startYear'])

In [5]:
cmu_movies['movie_name'] = cmu_movies['movie_name'].apply(lambda x : x.lower())

In [6]:
cmu_imdb_movies = cmu_movies.merge(imdb_movies,left_on='movie_name',right_on='primaryTitle')
cmu_imdb_movies = cmu_imdb_movies[~cmu_imdb_movies.duplicated(subset=['wikiID'],keep=False)]

In [7]:
# As IMDB has no missing values for the movie's year of release, for missing values in CMU we take the one from IMDB but keep information of months and days when available
cmu_imdb_movies.loc[cmu_imdb_movies['release_date'].isna(), 'release_date']=cmu_imdb_movies.loc[cmu_imdb_movies['release_date'].isna(), 'startYear']

In [8]:
cmu_movies = cmu_movies.drop('release_date',axis=1).merge(cmu_imdb_movies[['wikiID','averageRating','numVotes','release_date']],on='wikiID',how='inner')

## Let's put data in a more convenient format

### Release Dates

In [11]:
### Release Dates